## Imports

In [2]:
%load_ext autoreload
%autoreload 2
# add source folders to path
import sys,os
SRC = '..\\src'
src_folders = list(filter(lambda x: os.path.isdir(os.path.join(SRC,x)),os.listdir(SRC)))
[sys.path.append(os.path.join(SRC,folder)) for folder in src_folders]

# import custom libraries
from preamble import *
from helpers import save_joblib

# notebook imports

# custom file paths

# get data files
files_dict = walk_directory(raw_data)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Model outline
- Fuel Flows
    -   Quality, Fuel Analysis
- Air Flows
    - Air Splits
    - CO, NOx
    - Ambient Temp and Humidity
- Soot
    - Rate of development
    - Accumulation
    - Heat Transfer Effects
- Soot Blowers
    - Blowing Duration
    - Frequency?
    - Media
    - Effect on soot (should there be some long-term perminent degradation?)
    


In [ ]:
class PulverizedCoalSubcriticalBoiler():
    
    
    def __init__(self):
        # stub
        pass

    def simulate_next_time_step(self):
        return None
    
    def combustion_reaction():
        return None
